# CCC Clock Sensitivity Dashboard

Interactive sensitivity analysis for Computational Complexity Cosmology (CCC) clock experiments.

This notebook provides:
- Parameter space exploration for sets A, B, C
- Time-to-detect calculations and visualization
- SNR analysis across different configurations
- Acceptance criterion A1 validation

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

from metrology import CCCMetrology, PARAMETER_SETS, ABBASimulator
from protocol import ThetaLoop, ABBASequence, CCCProtocol

# Configure plotting
plt.style.use('default')
%matplotlib inline

## 1. Parameter Set Overview

In [ ]:
# Initialize metrology calculator
metrology = CCCMetrology(Gamma_Theta=1e-6, kappa=1.0)

# Display parameter sets
print("📊 CCC Clock Parameter Sets")
print("=" * 50)

param_data = []
for name, params in PARAMETER_SETS.items():
    R_op = metrology.compute_operational_curvature(params)
    param_data.append({
        'Set': name,
        'Description': params.description,
        'K_dot (bits/s)': f"{params.K_dot:.1e}",
        'S_loss_dot (bits/s)': f"{params.S_loss_dot:.1e}",
        'Temperature (K)': params.T,
        'R_op': f"{R_op:.2e}"
    })
    print(f"\n{params.name}:")
    print(f"  K̇ = {params.K_dot:.1e} bits/s")
    print(f"  Ṡ_loss = {params.S_loss_dot:.1e} bits/s")
    print(f"  T = {params.T} K")
    print(f"  R_op = {R_op:.2e}")

# Create parameter comparison table
df_params = pd.DataFrame(param_data)
print("\n📋 Parameter Summary Table:")
print(df_params.to_string(index=False))

## 2. Sensitivity Analysis

In [ ]:
# Define parameter ranges for sensitivity analysis
A_Sigma_range = np.logspace(-8, -4, 50)  # Loop areas from 1e-8 to 1e-4
sigma_0_values = [1e-17, 3e-18]  # Allan deviation coefficients
z_alpha = 3.0  # 3-sigma detection threshold

# Perform comprehensive sensitivity analysis
print("🔍 Performing sensitivity analysis...")
sensitivity_results = metrology.sensitivity_analysis(
    A_Sigma_range=A_Sigma_range,
    sigma_0_values=sigma_0_values,
    z_alpha=z_alpha
)

print(f"✅ Analysis complete. A1 criterion met: {sensitivity_results['acceptance_A1_met']}")

In [ ]:
# Create interactive time-to-detect plots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Parameter Set A', 'Parameter Set B', 'Parameter Set C', 'A1 Criterion Check'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

colors = ['blue', 'red']
sigma_labels = ['σ₀ = 1×10⁻¹⁷/√τ', 'σ₀ = 3×10⁻¹⁸/√τ']

# Plot for each parameter set
for i, (param_name, param_results) in enumerate(sensitivity_results['parameter_sets'].items()):
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    for j, sigma_0 in enumerate(sigma_0_values):
        tau_req_hours = param_results['tau_req_matrix'][j, :] / 3600  # Convert to hours
        
        fig.add_trace(
            go.Scatter(
                x=A_Sigma_range,
                y=tau_req_hours,
                mode='lines',
                name=f"{param_name}: {sigma_labels[j]}",
                line=dict(color=colors[j], width=2),
                showlegend=(i == 0)  # Only show legend for first subplot
            ),
            row=row, col=col
        )
    
    # Add 72-hour line (A1 criterion)
    fig.add_hline(y=72, line_dash="dash", line_color="green", 
                  annotation_text="72h (A1 limit)", row=row, col=col)
    
    fig.update_xaxes(type="log", title_text="Loop Area A_Σ", row=row, col=col)
    fig.update_yaxes(type="log", title_text="Time to Detect (hours)", row=row, col=col)

# A1 criterion summary in fourth subplot
a1_data = []
for param_name, param_results in sensitivity_results['parameter_sets'].items():
    for j, sigma_0 in enumerate(sigma_0_values):
        tau_req_min = np.min(param_results['tau_req_matrix'][j, :]) / 3600
        a1_met = tau_req_min <= 72
        a1_data.append({
            'Parameter Set': param_name,
            'σ₀': f"{sigma_0:.0e}",
            'Min τ_req (h)': tau_req_min,
            'A1 Met': a1_met
        })

# Create A1 summary bar chart
df_a1 = pd.DataFrame(a1_data)
for i, row in df_a1.iterrows():
    color = 'green' if row['A1 Met'] else 'red'
    fig.add_trace(
        go.Bar(
            x=[f"{row['Parameter Set']} ({row['σ₀']})"],
            y=[row['Min τ_req (h)']],
            marker_color=color,
            name=f"{row['Parameter Set']} {row['σ₀']}",
            showlegend=False
        ),
        row=2, col=2
    )

fig.add_hline(y=72, line_dash="dash", line_color="black", 
              annotation_text="A1 Limit", row=2, col=2)
fig.update_yaxes(type="log", title_text="Min Time to Detect (hours)", row=2, col=2)
fig.update_xaxes(title_text="Parameter Set (σ₀)", row=2, col=2)

fig.update_layout(
    height=800,
    title_text="CCC Clock Sensitivity Analysis - Time to Detect",
    showlegend=True
)

fig.show()

## 3. SNR Analysis

In [ ]:
# SNR analysis at different integration times
integration_times = [1*3600, 24*3600, 72*3600]  # 1 hour, 1 day, 3 days
time_labels = ['1 hour', '1 day', '3 days']

fig_snr = make_subplots(
    rows=1, cols=3,
    subplot_titles=time_labels,
    shared_yaxes=True
)

for t_idx, (tau, time_label) in enumerate(zip(integration_times, time_labels)):
    for param_name, param_results in sensitivity_results['parameter_sets'].items():
        for j, sigma_0 in enumerate(sigma_0_values):
            # Compute SNR for this integration time
            snr_values = []
            for A_Sigma in A_Sigma_range:
                params = PARAMETER_SETS[param_name]
                snr = metrology.compute_snr(params, A_Sigma, sigma_0, tau)
                snr_values.append(snr)
            
            fig_snr.add_trace(
                go.Scatter(
                    x=A_Sigma_range,
                    y=snr_values,
                    mode='lines',
                    name=f"{param_name} ({sigma_labels[j]})",
                    line=dict(width=2),
                    showlegend=(t_idx == 0)  # Only show legend for first subplot
                ),
                row=1, col=t_idx+1
            )
    
    # Add SNR = 3 line (detection threshold)
    fig_snr.add_hline(y=3, line_dash="dash", line_color="black", 
                      annotation_text="SNR = 3", row=1, col=t_idx+1)
    
    fig_snr.update_xaxes(type="log", title_text="Loop Area A_Σ", row=1, col=t_idx+1)
    if t_idx == 0:
        fig_snr.update_yaxes(type="log", title_text="Signal-to-Noise Ratio", row=1, col=t_idx+1)

fig_snr.update_layout(
    height=400,
    title_text="CCC Clock SNR Analysis at Different Integration Times",
    showlegend=True
)

fig_snr.show()

## 4. ABBA Simulation Demo

In [ ]:
# Demonstrate ABBA demodulation with sign flip test
print("🔄 ABBA Demodulation Simulation")
print("=" * 40)

# Initialize ABBA simulator
simulator = ABBASimulator(f_m=0.5, cycle_time=20.0)  # 0.5 Hz, 20s cycles

# Use parameter set A for demonstration
params_demo = PARAMETER_SETS['A']
A_Sigma_demo = 1e-6  # Example loop area
duration_demo = 600  # 10 minutes

# Simulate sign flip demonstration
sign_flip_demo = simulator.demonstrate_sign_flip(
    params_demo, A_Sigma_demo, duration=duration_demo
)

print(f"Sign flip detected: {sign_flip_demo['sign_flip_detected']}")
print(f"Signal ratio (reversed/normal): {sign_flip_demo['ratio']:.3f}")

# Plot ABBA traces
fig_abba = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Normal Loop - Raw Signal', 'Normal Loop - Demodulated',
                   'Reversed Loop - Raw Signal', 'Reversed Loop - Demodulated'),
    shared_xaxes=True
)

# Normal loop traces
normal_trace = sign_flip_demo['normal']
time_normal = normal_trace['time']
fig_abba.add_trace(
    go.Scatter(x=time_normal, y=normal_trace['total_signal'], 
               mode='lines', name='Raw Signal', line=dict(width=1)),
    row=1, col=1
)
fig_abba.add_trace(
    go.Scatter(x=time_normal, y=normal_trace['modulation'] * np.max(normal_trace['total_signal']) * 0.5, 
               mode='lines', name='Modulation', line=dict(width=2, color='red')),
    row=1, col=1
)

fig_abba.add_trace(
    go.Scatter(x=time_normal, y=normal_trace['demod_filtered'], 
               mode='lines', name='Demodulated', line=dict(width=2)),
    row=1, col=2
)

# Reversed loop traces
reversed_trace = sign_flip_demo['reversed']
time_reversed = reversed_trace['time']
fig_abba.add_trace(
    go.Scatter(x=time_reversed, y=reversed_trace['total_signal'], 
               mode='lines', name='Raw Signal (Rev)', line=dict(width=1)),
    row=2, col=1
)
fig_abba.add_trace(
    go.Scatter(x=time_reversed, y=reversed_trace['modulation'] * np.max(reversed_trace['total_signal']) * 0.5, 
               mode='lines', name='Modulation (Rev)', line=dict(width=2, color='red')),
    row=2, col=1
)

fig_abba.add_trace(
    go.Scatter(x=time_reversed, y=reversed_trace['demod_filtered'], 
               mode='lines', name='Demodulated (Rev)', line=dict(width=2)),
    row=2, col=2
)

fig_abba.update_xaxes(title_text="Time (s)")
fig_abba.update_yaxes(title_text="Signal Amplitude")
fig_abba.update_layout(
    height=600,
    title_text=f"ABBA Demodulation Demo - Sign Flip Test (Ratio: {sign_flip_demo['ratio']:.3f})",
    showlegend=False
)

fig_abba.show()

# Validate A3 criterion
if sign_flip_demo['sign_flip_detected']:
    print("\n✅ A3 Criterion: Sign flip under loop reversal confirmed!")
else:
    print("\n❌ A3 Criterion: Sign flip not detected (may need longer integration)")

## 5. Acceptance Criteria Summary

In [ ]:
# Comprehensive acceptance criteria check
print("📋 CCC Clock Acceptance Criteria Summary")
print("=" * 50)

# A1: τ_req ≤ 72h for at least one case at σ₀=3×10⁻¹⁸/√τ
a1_met = sensitivity_results['acceptance_A1_met']
print(f"\n✅ A1 - Time to detect ≤ 72h: {'PASS' if a1_met else 'FAIL'}")

if a1_met:
    # Find which cases meet A1
    for param_name, param_results in sensitivity_results['parameter_sets'].items():
        sigma_idx = 1  # σ₀ = 3×10⁻¹⁸
        tau_req_min = np.min(param_results['tau_req_matrix'][sigma_idx, :]) / 3600
        if tau_req_min <= 72:
            best_A_Sigma = A_Sigma_range[np.argmin(param_results['tau_req_matrix'][sigma_idx, :])]
            print(f"   Parameter set {param_name}: τ_req = {tau_req_min:.1f}h at A_Σ = {best_A_Sigma:.2e}")

# A3: ABBA traces show sign flip under loop reversal
a3_met = sign_flip_demo['sign_flip_detected']
print(f"\n✅ A3 - Sign flip under loop reversal: {'PASS' if a3_met else 'FAIL'}")
if a3_met:
    print(f"   Signal ratio (reversed/normal): {sign_flip_demo['ratio']:.3f}")

# Overall assessment
criteria_met = [a1_met, a3_met]  # A2 will be checked in bridge analysis notebook
overall_pass = all(criteria_met)

print(f"\n{'='*50}")
print(f"Overall Status: {'✅ READY FOR EXPERIMENT' if overall_pass else '❌ NEEDS IMPROVEMENT'}")
print(f"Criteria met: {sum(criteria_met)}/3 (A2 pending bridge analysis)")

# Generate summary table for export
summary_data = []
for param_name, param_results in sensitivity_results['parameter_sets'].items():
    R_op = param_results['R_op']
    for j, sigma_0 in enumerate(sigma_0_values):
        tau_req_min = np.min(param_results['tau_req_matrix'][j, :]) / 3600
        best_A_Sigma = A_Sigma_range[np.argmin(param_results['tau_req_matrix'][j, :])]
        
        summary_data.append({
            'Parameter_Set': param_name,
            'R_op': f"{R_op:.2e}",
            'sigma_0': f"{sigma_0:.0e}",
            'min_tau_req_hours': f"{tau_req_min:.1f}",
            'optimal_A_Sigma': f"{best_A_Sigma:.2e}",
            'A1_criterion_met': tau_req_min <= 72
        })

df_summary = pd.DataFrame(summary_data)
print("\n📊 Detailed Results Summary:")
print(df_summary.to_string(index=False))

# Save results for further analysis
df_summary.to_csv('../figures/sensitivity_analysis_results.csv', index=False)
print("\n💾 Results saved to: ../figures/sensitivity_analysis_results.csv")

## 6. Interactive Parameter Explorer

In [ ]:
# Create interactive parameter explorer
from ipywidgets import interact, FloatSlider, Dropdown, fixed
import ipywidgets as widgets

def interactive_sensitivity(Gamma_Theta=1e-6, A_Sigma=1e-6, sigma_0=3e-18, param_set='A'):
    """Interactive sensitivity analysis function."""
    
    # Initialize metrology with custom Gamma_Theta
    metrology_custom = CCCMetrology(Gamma_Theta=Gamma_Theta)
    params = PARAMETER_SETS[param_set]
    
    # Compute key metrics
    R_op = metrology_custom.compute_operational_curvature(params)
    tau_req = metrology_custom.compute_time_to_detect(params, A_Sigma, sigma_0)
    snr_1day = metrology_custom.compute_snr(params, A_Sigma, sigma_0, 24*3600)
    signal_amplitude = metrology_custom.compute_clock_observable(params, A_Sigma)
    
    # Display results
    print(f"Parameter Set: {params.name}")
    print(f"Γ_Θ = {Gamma_Theta:.1e}")
    print(f"A_Σ = {A_Sigma:.1e}")
    print(f"σ₀ = {sigma_0:.1e}/√τ")
    print(f"")
    print(f"R_op = {R_op:.2e}")
    print(f"Signal amplitude = {signal_amplitude:.2e}")
    print(f"τ_req = {tau_req/3600:.1f} hours")
    print(f"SNR (1 day) = {snr_1day:.2f}")
    print(f"")
    print(f"A1 criterion (τ_req ≤ 72h): {'✅ PASS' if tau_req <= 72*3600 else '❌ FAIL'}")
    
    return {
        'R_op': R_op,
        'tau_req_hours': tau_req/3600,
        'snr_1day': snr_1day,
        'signal_amplitude': signal_amplitude
    }

# Create interactive widget
print("🎛️ Interactive Parameter Explorer")
print("Adjust the parameters below to explore the sensitivity:")

interact(
    interactive_sensitivity,
    Gamma_Theta=FloatSlider(value=1e-6, min=1e-8, max=1e-4, step=1e-7, 
                           description='Γ_Θ', readout_format='.1e'),
    A_Sigma=FloatSlider(value=1e-6, min=1e-8, max=1e-4, step=1e-7, 
                       description='A_Σ', readout_format='.1e'),
    sigma_0=FloatSlider(value=3e-18, min=1e-19, max=1e-16, step=1e-19, 
                       description='σ₀', readout_format='.1e'),
    param_set=Dropdown(options=['A', 'B', 'C'], value='A', description='Parameter Set')
);